# Necessary Imports

In [208]:
import os
from sklearn import metrics
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import nltk
from nltk.metrics import ConfusionMatrix

import warnings
warnings.filterwarnings('ignore')

# Functions from starter_code.py

In [260]:
def evaluate(test_sentences, tagged_test_sentences, output_dict = False):
    gold = [str(tag) for sentence in test_sentences for token, tag in sentence]
    pred = [str(tag) for sentence in tagged_test_sentences for token, tag in sentence]
    return metrics.classification_report(gold, pred, output_dict = output_dict)

def get_token_tag_tuples(sent):
    return [nltk.tag.str2tuple(t) for t in sent.split()]

def get_tagged_sentences(text):
    sentences = list()

    blocks = text.split("======================================")
    for block in blocks:
        sents = block.split("\n\n")
        for sent in sents:
            sent = sent.replace("\n", "").replace("[", "").replace("]", "")
            if sent is not "":
                sentences.append(sent)
    return sentences

def load_treebank_splits(datadir):

    train = list()
    dev = list()
    test = list()

    print("Loading Penn Treebank Data...")
    print('`````````````````````````````````````')
    
    for subdir, dirs, files in os.walk(datadir):
        for filename in files:
            if filename.endswith(".pos"):
                filepath = subdir + os.sep + filename
                with open(filepath, "r") as fh:
                    text = fh.read()
                    if int(subdir.split(os.sep)[-1]) in range(0, 19):
                        train += get_tagged_sentences(text)

                    if int(subdir.split(os.sep)[-1]) in range(19, 22):
                        dev += get_tagged_sentences(text)
                    
                    if int(subdir.split(os.sep)[-1]) in range(22, 25):
                        test += get_tagged_sentences(text)

    print("Train set size: ", len(train))
    print("Dev set size: ", len(dev))
    print("Test set size: ", len(test))

    return train, dev, test

# Load Train, Dev and Test Datasets

In [248]:
# Set path for Datadir
datadir = os.path.join("data", "penn-treebank3-wsj", "wsj")
train, dev, test = load_treebank_splits(datadir)

Loading Penn Treebank Data...
`````````````````````````````````````
Train set size:  51681
Dev set size:  7863
Test set size:  9046


# Pre-Processing Helper Class

In [210]:
class PreProcess:
    # Constructor
    def __init__(self):
        # Special Tokens
        self.SOS = '<START>'
        self.EOS = '<STOP>'
        self.UNK = '<UNK>'

    def preprocess_dataset(self, 
                           dataset):
        preprocessed_ds = [([(self.SOS, self.SOS)] + get_token_tag_tuples(sentence) + [(self.EOS, self.EOS)]) for sentence in dataset]
        return preprocessed_ds

    def remove_start_stop(self, 
                          dataset):
        return [sentence[1:-1] for sentence in dataset]

    def get_wordlist(self, 
                     dataset):
        word_list = list()
        for sentence in dataset:
            word_list.append([word for word, tag in sentence])
        return word_list
    
    def get_taglist(self, 
                    sentence):
        tag_list = [tag for word, tag in sentence]
        return tag_list
    
    def flatten(self, 
                dataset, 
                preprocess):
        return [token for sentence in dataset for token in preprocess(sentence)]

In [212]:
pp = PreProcess()

# Gold Dev Dataset
gold_dev = pp.preprocess_dataset(dev)
# Gold Test Dataset
gold_test = pp.preprocess_dataset(test)

# Train Dataset with POS Tags
train_corpus = train
# Dev Dataset without POS Tags
dev_corpus = pp.get_wordlist(gold_dev)
# Test Dataset without POS Tags
test_corpus = pp.get_wordlist(gold_test)

# Gold Dev Dataset w/o <START> and <STOP>
gold_dev = pp.remove_start_stop(gold_dev)
# Gold Test Dataset w/o <START> and <STOP>
gold_test = pp.remove_start_stop(gold_test)

In [213]:
class BaselineTagger:
    # Constructor
    def __init__(self):
        self.corpus = list()  # training corpus

        self.most_frequent_table = (
            {}
        )  # key:val = word:most_freq_tag_of_such_word
        self.most_common_tag = (
            ""  # the most frequent tag no matter what word it is
        )

    def train(self, corpus):
        self.corpus = pp.preprocess_dataset(corpus)

        word_tag_frequent_table = {}
        tag_counts = {}

        # calculate all the word:tag count and individual tag count
        for sent in self.corpus:
            for token in sent:
                word, tag = token
                tag_frequency = word_tag_frequent_table.get(word, {})
                tag_frequency[tag] = tag_frequency.get(tag, 0) + 1
                word_tag_frequent_table[word] = tag_frequency
                tag_counts[tag] = tag_counts.get(tag, 0) + 1

        # calculate the most frequent tag for word
        for word, tag_freq in word_tag_frequent_table.items():
            self.most_frequent_table[word] = max(tag_freq.items(), key=lambda x: x[1])[0]
        # calculate the most frequent tag among all tags
        self.most_common_tag = max(tag_counts.items(), key=lambda x: x[1])[0]

    def predict(self, corpus):
        tags = list()
        for sent in tqdm(corpus, colour = 'red'):
            tag_seq = list()
            for word in sent:
                # if word in the dictionary, get the most frequent tag
                # if not just give most common tag
                tag = self.most_frequent_table.get(word, self.most_common_tag)
                tag_seq.append((word, tag))
            tags.append(tag_seq)
        return tags

In [214]:
class HMMTagger:
    # Constructor
    def __init__(self):
        # preprocessed and labeled corpus for the POS task
        self.corpus = list()
        # frequency of word tokens
        self.vocab = {}
        # all type of tags
        self.tag_list = list()
        # table for the transition prob of all combination of tag->tag in the corpus
        self.transition_table = list()
        # table for the emission prob of all combination of word:tag in the corpus
        self.emission_table = list()
        # the emission probability for <UNK>:tag => actually it is just a prob for every single tag
        self.unknown_emission_prob = {}

    def train(self, corpus, alpha=1):
        self.corpus = pp.preprocess_dataset(corpus)
        self.vocab = self.get_vocab(self.corpus)
        (
            self.transition_table,
            self.emission_table,
            self.unknown_emission_prob,
            self.tag_list,
        ) = self.build_tables(alpha)

        self.word2idx = {
            word: idx for idx, word in enumerate(self.vocab.keys())
        }
        self.tag2idx = {tag: idx for idx, tag in enumerate(self.tag_list)}
        self.idx2word = {
            idx: word for idx, word in enumerate(self.vocab.keys())
        }
        self.idx2tag = {idx: tag for idx, tag in enumerate(self.tag_list)}

    def build_tables(self, alpha=1):
        # key is prev tag and current tag, value is count
        transitions = self.get_transitions(self.corpus)
        # key is tag, token, value is count
        emissions = self.get_emissions(self.corpus)

        # store frequency of each tag
        tag_dict = self.get_tag_freq(self.corpus)
        tag_list = tag_dict.keys()

        transition_table = self.create_transition_table(
            transitions, tag_dict, tag_list, alpha
        )
        emission_table, unknown_emission_prob = self.create_emission_table(
            emissions, tag_dict, tag_list, self.vocab, alpha
        )

        return (
            transition_table,
            emission_table,
            unknown_emission_prob,
            tag_list,
        )

    def get_vocab(self, sents):
        vocab = {}
        for sent in sents:
            for token in sent:
                word = token[0]
                vocab[word] = vocab.get(word, 0) + 1
        return vocab

    def get_transitions(self, sents):
        transitions = {}
        for sent in sents:
            for i in range(1, len(sent)):
                bigram_tags = (sent[i - 1][1], sent[i][1])
                transitions[bigram_tags] = transitions.get(bigram_tags, 0) + 1
        return transitions

    def get_emissions(self, sents):
        emissions = {}
        for sent in sents:
            for token_tag_pair in sent:
                emissions[token_tag_pair] = (emissions.get(token_tag_pair, 0) + 1)
        return emissions

    def get_tag_freq(self, sents):
        tag_dict = {}
        for sent in sents:
            for _, tag in sent:
                tag_dict[tag] = tag_dict.get(tag, 0) + 1
        return tag_dict

    def create_transition_table(self, transitions, tag_dict, tags, alpha=1):
        transition_table = list()  # 2-dim list
        for prev_tag in tags:
            prob_list = list()
            for current_tag in tags:
                prev_count = tag_dict.get(prev_tag, 0)
                bigram_count = transitions.get((prev_tag, current_tag), 0)
                prob = (bigram_count + alpha) / (prev_count + (alpha * len(tags)))
                prob_list.append(np.log(prob))
            transition_table.append(prob_list)
        return transition_table

    def create_emission_table(self, emissions, tag_dict, tags, vocab, alpha):
        emission_table = list()  # 2-dim list
        unknown_emission_prob = {}
        total_tag_counts = sum(tag_dict.values())
        for tag in tags:
            prob_list = list()
            tag_count = tag_dict.get(tag, 0)
            for word in vocab.keys():
                word_tag_count = emissions.get((word, tag), 0)
                prob = (word_tag_count + alpha) / (tag_count + (alpha * len(tags)))
                prob_list.append(np.log(prob))
            emission_table.append(prob_list)
            unknown_emission_prob[tag] = (tag_count + alpha) / (total_tag_counts + (alpha * len(tags)))
        return emission_table, unknown_emission_prob

    def viterbi_algorithm(self, sent):
        tags = list()
        viterbi_matrix = list()

        # Initial step
        initial = list()  # empty array for start token
        viterbi_matrix.append(initial)
        first_token = sent[1]
        first_token_scores = list()
        for i, tag in enumerate(self.tag_list):
            transition_prob = self.transition_table[self.tag2idx[pp.SOS]][i]
            emission_prob = self.unknown_emission_prob[tag]
            if first_token in self.word2idx.keys():
                emission_prob = self.emission_table[i][self.word2idx[first_token]]
            # calculate all the tag start from the start token
            first_token_scores.append((self.tag2idx[pp.SOS], transition_prob + emission_prob))
        viterbi_matrix.append(first_token_scores)

        # recursive step
        for t, token in enumerate(sent):
            if t <= 1:
                continue
            max_scores = list()
            for i, tag in enumerate(self.tag_list):
                max_score = float("-inf")
                candidate = None
                emission_prob = self.unknown_emission_prob[tag]
                if token in self.word2idx.keys():
                    emission_prob = self.emission_table[i][self.word2idx[token]]
                # go through every previous score that already be calculated in the viterbi matrix
                for j, score in enumerate(viterbi_matrix[t - 1]):
                    _, prev_max_log_prob = score
                    transition_prob = self.transition_table[j][i]
                    new_score = (emission_prob + transition_prob + prev_max_log_prob)
                    if new_score > max_score:
                        max_score = new_score
                        candidate = j
                max_scores.append((candidate, max_score))
            viterbi_matrix.append(max_scores)

        # start with the stop tag
        max_tag = self.tag2idx[pp.EOS]
        tags.append((pp.EOS, self.idx2tag[max_tag]))

        # find best path in viterbi matrix
        for i in reversed(range(1, len(viterbi_matrix))):
            max_tag = viterbi_matrix[i][max_tag][0]
            tags.append((sent[i - 1], self.idx2tag[max_tag]))

        # since it is found backward, we need to reverse it
        tags.reverse()
        return tags

    def predict(self, corpus):
        all_tags = list()
        for sent in tqdm(corpus, colour = 'green'):
            prediction_tags = self.viterbi_algorithm(sent)
            all_tags.append(prediction_tags)
        return all_tags

# Debug 

In [237]:
def debug(alpha = 1):
    print('`````````````````````````````````````')
    print('DEBUG CODE...')
    print('`````````````````````````````````````')
    
    train_samples = [
        "The/DT House/NNP joined/VBD the/DT Senate/NNP in/IN making/VBG federal/JJ reparations/NNS for/IN Japanese-Americans/NNPS"
    ]
    test_samples = [
        "alsjfla the askdmc and djdsaas in making",
        "vsacs the House and djdsaas in giuhun",
        "The House joined  the Senate in making  federal reparations for Japanese-Americans",
    ]
    tagger = HMMTagger()
    tagger.train(train_samples, alpha)
    hmm_y = tagger.predict(test_samples)

    b_tagger = BaselineTagger()
    b_tagger.train(train_corpus)
    b_y = b_tagger.predict(test_samples)

    hmm_y = pp.remove_start_stop(hmm_y)
    b_y = pp.remove_start_stop(b_y)
    result = evaluate(hmm_y, b_y)
    print(result)

In [238]:
debug()

`````````````````````````````````````
DEBUG CODE
`````````````````````````````````````


100%|██████████████████████████████████████████| 3/3 [00:00<00:00, 55431.33it/s]

              precision    recall  f1-score   support

           :       0.00      0.00      0.00         0
          DT       0.42      0.15      0.23        52
          FW       0.00      0.00      0.00         0
          NN       0.00      0.00      0.00         0
         NNP       0.00      0.00      0.00        52
         SYM       0.00      0.00      0.00         0
         VBD       0.00      0.00      0.00        49
         VBZ       0.00      0.00      0.00         0

    accuracy                           0.05       153
   macro avg       0.05      0.02      0.03       153
weighted avg       0.14      0.05      0.08       153



# Baseline Tagger Results on Dev and Test Set

In [249]:
def baseline():
    # Train Set
    baseline_tagger = BaselineTagger()
    baseline_tagger.train(train_corpus)

    # Dev Set
    prediction_dev = baseline_tagger.predict(dev_corpus)
    prediction_dev = pp.remove_start_stop(prediction_dev)
    dev_res = evaluate(gold_dev, prediction_dev)
    print('`````````````````````````````````````')
    print('Baseline: DEV SET...')
    print('`````````````````````````````````````')
    print(dev_res)

    # Test Set
    prediction_test = baseline_tagger.predict(test_corpus)
    prediction_test = pp.remove_start_stop(prediction_test)
    test_res = evaluate(gold_test, prediction_test)
    print('`````````````````````````````````````')
    print('Baseline: TEST SET...')
    print('`````````````````````````````````````')
    print(test_res)

In [250]:
baseline()

100%|███████████████████████████████████| 7863/7863 [00:00<00:00, 226442.82it/s]


`````````````````````````````````````
Baseline: DEV SET...
`````````````````````````````````````
              precision    recall  f1-score   support

           #       1.00      1.00      1.00        31
           $       1.00      1.00      1.00      1248
          ''       1.00      0.98      0.99      1168
           (       1.00      1.00      1.00       244
           )       1.00      1.00      1.00       244
           ,       1.00      1.00      1.00      7931
           .       1.00      1.00      1.00      6125
           :       1.00      1.00      1.00       775
          CC       0.99      0.99      0.99      3777
          CD       0.99      0.90      0.94      5766
          DT       0.99      0.98      0.99     12639
          EX       0.86      1.00      0.92       133
          FW       0.53      0.40      0.45        25
          IN       0.95      0.98      0.96     15497
       IN|RB       0.00      0.00      0.00         1
          JJ       0.88      0.84     

100%|███████████████████████████████████| 9046/9046 [00:00<00:00, 207601.55it/s]


`````````````````````````````````````
Baseline: TEST SET...
`````````````````````````````````````
              precision    recall  f1-score   support

           #       1.00      1.00      1.00        22
           $       1.00      1.00      1.00      1138
          ''       1.00      0.99      1.00      1423
           (       1.00      1.00      1.00       249
           )       1.00      1.00      1.00       252
           ,       1.00      1.00      1.00      9056
           .       1.00      1.00      1.00      7035
           :       1.00      1.00      1.00       983
          CC       1.00      1.00      1.00      4289
          CD       0.99      0.90      0.94      6023
          DT       0.99      0.99      0.99     14946
          EX       0.89      1.00      0.94       174
          FW       0.35      0.21      0.26        38
          IN       0.94      0.98      0.96     18147
          JJ       0.88      0.86      0.87     10704
         JJR       0.66      0.95    

# HMM Tagger Results on Dev and Test Set with Alpha = 1

In [242]:
def hmm(alpha = 1):
    # Train Set
    hmm_tagger = HMMTagger()
    hmm_tagger.train(train_corpus, alpha)

    # Dev Set
    prediction_dev = hmm_tagger.predict(dev_corpus)
    prediction_dev = pp.remove_start_stop(prediction_dev)
    dev_res = evaluate(gold_dev, prediction_dev)
    print('`````````````````````````````````````')
    print('HMM: DEV SET --> Alpha = {0}'.format(alpha))
    print('`````````````````````````````````````')
    print(dev_res)

    # Test Set
    prediction_test = hmm_tagger.predict(test_corpus)
    prediction_test = pp.remove_start_stop(prediction_test)    
    test_res = evaluate(gold_test, prediction_test)
    print('`````````````````````````````````````')
    print('HMM: TEST SET --> Alpha = {0}'.format(alpha))
    print('`````````````````````````````````````')
    print(test_res)

    # Plot Confusion Matrix
    cm = ConfusionMatrix(pp.flatten(gold_test, pp.get_taglist), pp.flatten(prediction_test, pp.get_taglist))
    print('`````````````````````````````````````')
    print('CONFUSION MATRIX --> Alpha = {0}'.format(alpha))
    print('`````````````````````````````````````')
    print(cm)

In [220]:
hmm()

100%|███████████████████████████████████████| 7863/7863 [01:41<00:00, 77.25it/s]


`````````````````````````````````````
HMM: DEV SET --> Alpha = 1
`````````````````````````````````````
              precision    recall  f1-score   support

           #       0.26      1.00      0.41        31
           $       0.87      1.00      0.93      1248
          ''       0.87      0.98      0.92      1168
           (       0.86      0.97      0.91       244
           )       0.80      0.96      0.87       244
           ,       0.99      0.99      0.99      7931
           .       0.97      1.00      0.99      6125
           :       0.96      0.90      0.93       775
          CC       0.99      0.91      0.95      3777
          CD       0.98      0.85      0.91      5766
          DT       0.96      0.96      0.96     12639
          EX       0.45      0.98      0.62       133
          FW       0.00      0.40      0.00        25
          IN       0.97      0.90      0.94     15497
       IN|RB       0.00      0.00      0.00         1
          JJ       0.86      0.6

100%|███████████████████████████████████████| 9046/9046 [01:57<00:00, 76.75it/s]


`````````````````````````````````````
HMM: TEST SET --> Alpha = 1
`````````````````````````````````````
              precision    recall  f1-score   support

           #       0.24      1.00      0.38        22
           $       0.83      1.00      0.91      1138
          ''       0.89      0.98      0.93      1423
           (       0.86      0.95      0.90       249
           )       0.75      0.94      0.83       252
           ,       0.99      0.99      0.99      9056
           .       0.98      1.00      0.99      7035
           :       0.96      0.90      0.93       983
          CC       1.00      0.92      0.95      4289
          CD       0.98      0.83      0.90      6023
          DT       0.96      0.96      0.96     14946
          EX       0.44      0.99      0.61       174
          FW       0.00      0.37      0.00        38
          IN       0.97      0.91      0.94     18147
       IN|RB       0.00      0.00      0.00         0
          JJ       0.88      0.

# Hyper-Parameter Tuning on Dev Set with diff. values of Alpha

In [268]:
def hmm_dev_set(alpha = 1):
    hmm_tagger = HMMTagger()
    hmm_tagger.train(train_corpus, alpha)
    
    # Dev Set
    prediction_dev = hmm_tagger.predict(dev_corpus)
    prediction_dev = pp.remove_start_stop(prediction_dev)
    dev_res = evaluate(gold_dev, prediction_dev, output_dict = True)
    return dev_res

In [287]:
def hyperparam_tuning_dev_set():
    alphas = np.logspace(0, -9, 10)
    results_dataframe = pd.DataFrame(alphas, 
                                     columns = ['alpha']
                                    )
    
    accuracy = list()
    macro_precision = list()
    macro_recall = list()
    macro_f1 = list()
    weighted_precision = list()
    weighted_recall = list()
    weighted_f1 = list()
    
    for alpha in alphas:
        print('`````````````````````````````````````')
        print('Alpha = {0}'.format(alpha))
        classification_report = hmm_dev_set(alpha)
        print('`````````````````````````````````````')
        accuracy.append(round(classification_report['accuracy'], 5))
        macro_precision.append(round(classification_report['macro avg']['precision'], 5))
        macro_recall.append(round(classification_report['macro avg']['recall'], 5))
        macro_f1.append(round(classification_report['macro avg']['f1-score'], 5))
        weighted_precision.append(round(classification_report['weighted avg']['precision'], 5))
        weighted_recall.append(round(classification_report['weighted avg']['recall'], 5))
        weighted_f1.append(round(classification_report['weighted avg']['f1-score'], 5))

    results_dataframe['accuracy'] = accuracy
    results_dataframe['macro_precision'] = macro_precision
    results_dataframe['macro_recall'] = macro_recall
    results_dataframe['macro_f1'] = macro_f1
    results_dataframe['weighted_precision'] = weighted_precision
    results_dataframe['weighted_recall'] = weighted_recall
    results_dataframe['weighted_f1'] = weighted_f1
    
    results_dataframe.to_csv('result_report.csv', index = False)
    
    return results_dataframe

df = hyperparam_tuning_dev_set()

`````````````````````````````````````
Alpha = 1.0


100%|███████████████████████████████████████| 7863/7863 [01:40<00:00, 78.06it/s]


`````````````````````````````````````
`````````````````````````````````````
Alpha = 0.1


100%|███████████████████████████████████████| 7863/7863 [01:44<00:00, 75.58it/s]


`````````````````````````````````````
`````````````````````````````````````
Alpha = 0.01


...

`````````````````````````````````````
`````````````````````````````````````
Alpha = 1e-08


100%|███████████████████████████████████████| 7863/7863 [01:42<00:00, 76.97it/s]


`````````````````````````````````````
`````````````````````````````````````
Alpha = 1e-09


100%|███████████████████████████████████████| 7863/7863 [01:42<00:00, 77.00it/s]


`````````````````````````````````````


In [289]:
print('``````````````````````````````````````````````````````````')
print('Hyper-Parameter Tuning Results with diff. Values of Alpha')
print('``````````````````````````````````````````````````````````')
df.head(10)

``````````````````````````````````````````````````````````
Hyper-Parameter Tuning Results with diff. Values of Alpha
``````````````````````````````````````````````````````````


alpha  accuracy  macro_precision  macro_recall  macro_f1  \
0  1.000000e+00   0.83797          0.50493       0.58993   0.50834   
1  1.000000e-01   0.92474          0.46017       0.52691   0.47118   
2  1.000000e-02   0.94198          0.48299       0.51756   0.49066   
3  1.000000e-03   0.94878          0.50768       0.52531   0.51285   
4  1.000000e-04   0.94899          0.61562       0.63055   0.61964   
5  1.000000e-05   0.94900          0.65940       0.67190   0.66216   
6  1.000000e-06   0.94898          0.65940       0.67188   0.66215   
7  1.000000e-07   0.94898          0.65940       0.67188   0.66215   
8  1.000000e-08   0.94898          0.65940       0.67188   0.66215   
9  1.000000e-09   0.94898          0.65940       0.67188   0.66215   

   weighted_precision  weighted_recall  weighted_f1  
0             0.93696          0.83797      0.88052  
1             0.94865          0.92474      0.93574  
2             0.95096          0.94198      0.94611  
3             0.95042          0.94878      0.94933  
4             0.95033          0.94899      0.94940  
5             0.95028          0.94900      0.94938  
6             0.95026          0.94898      0.94936  
7             0.95026          0.94898      0.94936  
8             0.95026          0.94898      0.94936  
9             0.95026          0.94898      0.94936

# Results with Best Alpha = 1e-5

In [243]:
hmm(alpha = 1e-5)

100%|███████████████████████████████████████| 7863/7863 [01:41<00:00, 77.24it/s]


`````````````````````````````````````
HMM: DEV SET --> Alpha = 1e-05
`````````````````````````````````````
              precision    recall  f1-score   support

           #       1.00      1.00      1.00        31
           $       0.98      1.00      0.99      1248
          ''       0.99      1.00      1.00      1168
           (       1.00      1.00      1.00       244
           )       1.00      1.00      1.00       244
           ,       0.99      1.00      1.00      7931
           .       0.99      1.00      1.00      6125
           :       1.00      0.99      1.00       775
          CC       1.00      0.99      1.00      3777
          CD       0.99      0.95      0.97      5766
          DT       0.96      0.99      0.97     12639
          EX       0.98      0.98      0.98       133
          FW       0.44      0.32      0.37        25
          IN       0.97      0.97      0.97     15497
       IN|RB       0.00      0.00      0.00         1
          JJ       0.88     

100%|███████████████████████████████████████| 9046/9046 [01:58<00:00, 76.63it/s]


`````````````````````````````````````
HMM: TEST SET --> Alpha = 1e-05
`````````````````````````````````````
              precision    recall  f1-score   support

           #       1.00      1.00      1.00        22
           $       0.97      1.00      0.98      1138
          ''       1.00      1.00      1.00      1423
           (       1.00      1.00      1.00       249
           )       1.00      1.00      1.00       252
           ,       0.99      1.00      1.00      9056
           .       0.99      1.00      1.00      7035
           :       1.00      1.00      1.00       983
          CC       1.00      1.00      1.00      4289
          CD       0.99      0.93      0.96      6023
          DT       0.96      0.99      0.97     14946
          EX       0.97      0.99      0.98       174
          FW       0.50      0.34      0.41        38
          IN       0.97      0.97      0.97     18147
       IN|RB       0.00      0.00      0.00         0
          JJ       0.89    